In [94]:
#Dependencies
import pandas as pd
import gmaps
# Dependencies
import requests
import json

# Google developer API key
import os
g_key = os.environ.get('GOOGLE_API_KEY')

In [95]:
data_file = 'Output/concated_df.csv'

data_file_df = pd.read_csv(data_file)
data_file_df.head()

,Country,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual,year
0,Switzerland,1,7.587,1.396510,1.349510,0.941430,0.665570,0.419780,0.296780,2.517380,2015
1,Finland,1,7.769,1.340000,1.587000,0.986000,0.596000,0.393000,0.153000,2.714000,2019
2,Finland,1,7.632,1.305000,1.592000,0.874000,0.681000,0.393000,0.202000,2.585000,2018
3,Norway,1,7.537,1.616463,1.533524,0.796667,0.635423,0.315964,0.362012,2.277027,2017
4,Denmark,1,7.526,1.441780,1.163740,0.795040,0.579410,0.444530,0.361710,2.739390,2016


In [96]:
# base_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
#     'address={0}&key={1}').format(target_city, g_key)

lat= []
lon =[]

countries = top_10_df["Country"]

#processing starting from 0
count = 1

# use iterrows to iterate through pandas dataframe
for country in countries:

    try:
        # Target city
        target_city = country

        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_city, g_key)

        # Run a request to endpoint and convert result to json
        geo_data = requests.get(target_url).json()

        # Print the json
        #print(geo_data)
    
        # Print the json (pretty printed)
        #print(json.dumps(geo_data, indent=4, sort_keys=True))
    
        # Extract latitude and longitude
        lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        #top_10_df_2015['Lat'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        #top_10_df_2015['Lon'] = geo_data["results"][0]["geometry"]["location"]["lng"]
        
        #loop to the next one
        count += 1
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [82]:
top_10_df["lat"] = lat
top_10_df["lon"] = lon

In [83]:
#display Dataframe (Lon and Lat)
top_10_df.head()

,Country,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual,year,lat,lon
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015,46.818188,8.227512
20,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015,56.130366,-106.346771
27,Finland,6,7.406,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955,2015,61.924110,25.748151
30,Netherlands,7,7.378,1.32944,1.28017,0.89284,0.61576,0.31814,0.47610,2.46570,2015,52.132633,5.291266
15,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015,60.472024,8.468946


In [84]:
top_10_df=top_10_df.sort_values(by=['year'])

In [85]:
#Top 10 by Region
import plotly.express as px
fig = px.scatter_geo(top_10_df, lat="lat",lon="lon", color="Happiness_Score", hover_name="Country" , 
                     text="Happiness_Rank", animation_frame="year",projection="natural earth",
                     size='Happiness_Score')

fig.show()

In [86]:
import plotly.io as pio
pio.write_html(fig, 'top10.html')

In [87]:
#filter the data by top 10 country
data_file_df.head()

,Country,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual,year
0,Switzerland,1,7.587,1.396510,1.349510,0.941430,0.665570,0.419780,0.296780,2.517380,2015
1,Finland,1,7.769,1.340000,1.587000,0.986000,0.596000,0.393000,0.153000,2.714000,2019
2,Finland,1,7.632,1.305000,1.592000,0.874000,0.681000,0.393000,0.202000,2.585000,2018
3,Norway,1,7.537,1.616463,1.533524,0.796667,0.635423,0.315964,0.362012,2.277027,2017
4,Denmark,1,7.526,1.441780,1.163740,0.795040,0.579410,0.444530,0.361710,2.739390,2016


In [88]:
data_file_df["year"].value_counts()

2015    158
2016    157
2019    156
2018    156
2017    155
Name: year, dtype: int64

In [89]:
#Find the last ranking of the year
data_file_df.groupby(['year'], as_index=False)['Happiness_Rank'].max()

,year,Happiness_Rank
0,2015,158
1,2016,157
2,2017,155
3,2018,156
4,2019,156


In [90]:

#filter the data by bottom 10 country
bottom_10_df = data_file_df.loc[data_file_df["Happiness_Rank"] >= 145 ]
bottom_10_df = bottom_10_df.drop(bottom_10_df[(data_file_df["year"]==2015) & (data_file_df["Happiness_Rank"]<=148)].index)
bottom_10_df = bottom_10_df.drop(bottom_10_df[(data_file_df["year"]==2018) & (data_file_df["Happiness_Rank"]<=145)].index)
bottom_10_df = bottom_10_df.drop(bottom_10_df[(data_file_df["year"]==2019) & (data_file_df["Happiness_Rank"]<=145)].index)
bottom_10_df = bottom_10_df.drop(bottom_10_df[(data_file_df["year"]==2016) & (data_file_df["Happiness_Rank"]<=147)].index)

<ipython-input-90-4fde3bb6b672>:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-90-4fde3bb6b672>:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-90-4fde3bb6b672>:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-90-4fde3bb6b672>:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [91]:
bottom_10_df["year"].value_counts()

2019    11
2018    11
2017    11
2016    10
2015    10
Name: year, dtype: int64

In [92]:
bottom_10_df.head()

,Country,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual,year
724,Haiti,145,3.603,0.368610,0.640450,0.277321,0.030370,0.099872,0.489204,1.697168,2017
726,Zimbabwe,146,3.663,0.366000,1.114000,0.433000,0.361000,0.089000,0.151000,1.149000,2019
728,Yemen,146,3.593,0.591683,0.935382,0.310081,0.249464,0.056767,0.104125,1.345601,2017
729,Botswana,146,3.590,1.017000,1.174000,0.417000,0.557000,0.092000,0.042000,0.291000,2018
730,South Sudan,147,3.591,0.397249,0.601323,0.163486,0.147062,0.116794,0.285671,1.879567,2017


In [93]:
base_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, g_key)

lat= []
lon =[]

countries = bottom_10_df["Country"]

#processing starting from 0
count = 1

# use iterrows to iterate through pandas dataframe
for country in countries:

    try:
        # Target city
        target_city = country

        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_city, g_key)

        # Run a request to endpoint and convert result to json
        geo_data = requests.get(target_url).json()

        # Print the json
        #print(geo_data)
    
        # Print the json (pretty printed)
        #print(json.dumps(geo_data, indent=4, sort_keys=True))
    
        # Extract latitude and longitude
        lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        #top_10_df_2015['Lat'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        #top_10_df_2015['Lon'] = geo_data["results"][0]["geometry"]["location"]["lng"]
        
        #loop to the next one
        count += 1
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [73]:
bottom_10_df["lat"] = lat
bottom_10_df["lon"] = lon

In [74]:
bottom_10_df=bottom_10_df.sort_values(by=['year','Happiness_Score'])

In [75]:
bottom_10_df

,Country,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Trust_(Government_Corruption),Generosity,Dystopia_Residual,year,lat,lon
781,Togo,158,2.839,0.208680,0.139950,0.284430,0.364530,0.107310,0.166810,1.567260,2015,8.619543,0.824782
780,Burundi,157,2.905,0.015300,0.415870,0.223960,0.118500,0.100620,0.197270,1.833020,2015,-3.373056,29.918886
776,Syria,156,3.006,0.663200,0.474890,0.721930,0.156840,0.189060,0.471790,0.328580,2015,34.802075,38.996815
774,Benin,155,3.340,0.286650,0.353860,0.319100,0.484500,0.080100,0.182600,1.633280,2015,9.307690,2.315834
767,Rwanda,154,3.465,0.222080,0.773700,0.428640,0.592010,0.551910,0.226280,0.670420,2015,-1.940278,29.873888
762,Afghanistan,153,3.575,0.319820,0.302850,0.303350,0.234140,0.097190,0.365100,1.952100,2015,33.939110,67.709953
759,Burkina Faso,152,3.587,0.258120,0.851880,0.271250,0.394930,0.128320,0.217470,1.464940,2015,12.238333,-1.561593
750,Ivory Coast,151,3.655,0.465340,0.771150,0.151850,0.468660,0.179220,0.201650,1.417230,2015,7.539989,-5.547080
745,Guinea,150,3.656,0.174170,0.464750,0.240090,0.377250,0.121390,0.286570,1.991720,2015,9.945587,-9.696645
743,Chad,149,3.667,0.341930,0.760620,0.150100,0.235010,0.052690,0.183860,1.942960,2015,15.454166,18.732207


In [76]:
#Bottom 10 by Region
fig = px.scatter_geo(bottom_10_df, lat="lat",lon="lon", color="Happiness_Score", hover_name="Country",
                     text="Happiness_Rank", animation_frame="year",projection="natural earth",size="Happiness_Score")

fig.show()

In [77]:
pio.write_html(fig, 'bottom10.html')